In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statistics as stat
import scipy as sci
pd.options.mode.chained_assignment = None

In [3]:
path = 'C:/Users/night/Documents/Out of the Park Developments/OOTP Baseball 22/saved_games/New Game.lg/import_export/csv/'
ranks = pd.read_csv(path + 'players_value.csv')
players = pd.read_csv(path + 'players.csv', low_memory = False)
teams = pd.read_csv(path + 'teams.csv', low_memory = False)
status = pd.read_csv(path + 'players_roster_status.csv', low_memory = False)
playerbatting = pd.read_csv(path + 'players_game_batting.csv', low_memory = False)
playerpitches = pd.read_csv(path + 'players_game_pitching_stats.csv', low_memory = False)
players = players[['player_id','team_id','league_id','first_name', 'last_name', 'age']]
players['player'] = players['first_name'] + ' ' + players['last_name']
players = players.drop(['first_name', 'last_name'], axis = 1)

teams = teams[['team_id','name','nickname']]
teams['team'] = teams['name'] + ' ' + teams['nickname']
teams = teams.drop(['name','nickname'], axis = 1)

bats = playerbatting[['player_id', 'team_id', 'league_id', 'game_id']]
bats = bats.groupby(['player_id', 'team_id', 'league_id']).count().reset_index()
bats = bats.rename(columns = {'game_id':'games'})
bats = bats[bats['games'] > 15]

arms = playerpitches[['player_id', 'team_id', 'league_id', 'game_id']]
arms = arms.groupby(['player_id', 'team_id', 'league_id']).count().reset_index()
arms = arms.rename(columns = {'game_id':'games'})
arms = arms[arms['games'] > 15]

criteria = pd.concat([bats,arms], ignore_index = True).drop_duplicates(subset='player_id', keep='first')
status = pd.merge(pd.merge(players, teams, on = 'team_id'), criteria, on = ['player_id', 'team_id', 'league_id']).dropna()
allranks = pd.merge(ranks, status, on=['player_id', 'team_id', 'league_id'])
allranks = allranks.rename(columns = {'overall_sp':'SP',
                                                'overall_rp':'RP',
                                                'overall_c': 'C',
                                                'overall_1b': '1B',
                                                'overall_2b': '2B',
                                                'overall_3b': '3B',
                                                'overall_ss': 'SS',
                                                'overall_lf': 'LF',
                                                'overall_cf': 'CF',
                                                'overall_rf': 'RF'})

ml_ranks = allranks[allranks['league_id'] == 145]
ml_ranks['positional_overall'] = ml_ranks[['SP','RP','C','1B','2B','3B','SS','LF','CF','RF']].max(axis=1)
ml_ranks['optimal_position'] = ml_ranks[['SP','RP','C','1B','2B','3B','SS','LF','CF','RF']].idxmax(axis=1)

C:\Users\night\AppData\Local\Temp\ipykernel_820\1362384380.py:42: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ml_ranks['optimal_position'] = ml_ranks[['SP','RP','C','1B','2B','3B','SS','LF','CF','RF']].idxmax(axis=1)


In [28]:
def dataset(league, league_id):
    global ml_ranks
    global norm_batting
    global norm_pitching
    global norm_ml

    path = 'C:/Users/night/Documents/Out of the Park Developments/OOTP Baseball 23/saved_games/' + league + '.lg/import_export/csv/'    
    ranks = pd.read_csv(path + 'players_value.csv')
    players = pd.read_csv(path + 'players.csv', low_memory = False)
    teams = pd.read_csv(path + 'teams.csv', low_memory = False)
    status = pd.read_csv(path + 'players_roster_status.csv', low_memory = False)
    playerbatting = pd.read_csv(path + 'players_game_batting.csv', low_memory = False)
    playerpitches = pd.read_csv(path + 'players_game_pitching_stats.csv', low_memory = False)

    players = players[['player_id','team_id','league_id','first_name', 'last_name', 'age']]
    players['player'] = players['first_name'] + ' ' + players['last_name']
    players = players.drop(['first_name', 'last_name'], axis = 1)

    teams = teams[['team_id','name','nickname']]
    teams['team'] = teams['name'] + ' ' + teams['nickname']
    teams = teams.drop(['name','nickname'], axis = 1)

    bats = playerbatting[['player_id', 'team_id', 'league_id', 'game_id']]
    bats = bats.groupby(['player_id', 'team_id', 'league_id']).count().reset_index()
    bats = bats.rename(columns = {'game_id':'games'})
    bats = bats[bats['games'] > 15]

    arms = playerpitches[['player_id', 'team_id', 'league_id', 'game_id']]
    arms = arms.groupby(['player_id', 'team_id', 'league_id']).count().reset_index()
    arms = arms.rename(columns = {'game_id':'games'})
    arms = arms[arms['games'] > 15]

    criteria = pd.concat([bats,arms], ignore_index = True).drop_duplicates(subset='player_id', keep='first')

    status = pd.merge(pd.merge(players, teams, on = 'team_id'), criteria, on = ['player_id', 'team_id', 'league_id']).dropna()
    allranks = pd.merge(ranks, status, on=['player_id', 'team_id', 'league_id'])
    allranks = allranks.rename(columns = {'overall_sp':'SP',
                                                    'overall_rp':'RP',
                                                    'overall_c': 'C',
                                                    'overall_1b': '1B',
                                                    'overall_2b': '2B',
                                                    'overall_3b': '3B',
                                                    'overall_ss': 'SS',
                                                    'overall_lf': 'LF',
                                                    'overall_cf': 'CF',
                                                    'overall_rf': 'RF'})

    ml_ranks = allranks[allranks['league_id'] == league_id]
    ml_ranks['positional_overall'] = ml_ranks[['SP','RP','C','1B','2B','3B','SS','LF','CF','RF']].max(axis=1)
    ml_ranks['optimal_position'] = ml_ranks[['SP','RP','C','1B','2B','3B','SS','LF','CF','RF']].idxmax(axis=1)
    
    eval_cols = ['player',
    'team',
    'age',
    'position',
    'optimal_position',
    'oa',
    'pot',
    'overall_value',
    'positional_overall',
    'talent_value',
    'running_value',
    'stealing_value',
    'ratings_value',
    'season_performance']
    
    conditions = [
        ml_ranks['position'] == 1,
        ml_ranks['position'] == 2,
        ml_ranks['position'] == 3,
        ml_ranks['position'] == 4,
        ml_ranks['position'] == 5,
        ml_ranks['position'] == 6,
        ml_ranks['position'] == 7,
        ml_ranks['position'] == 8,
        ml_ranks['position'] == 9,
        ml_ranks['position'] == 10,
    ]

    choices = [
        'P',
        'C',
        '1B',
        '2B',
        '3B',
        'SS',
        'LF',
        'CF',
        'RF',
        'DH'
    ]
    
    ml_ranks = ml_ranks[eval_cols].set_index('player')
    ml_ranks['position'] = np.select(conditions, choices)

    ml_pitchers = ml_ranks[ml_ranks['position'] == 'P']
    ml_batters = ml_ranks[ml_ranks['position'] != 'P']
    
    ml_batters = ml_batters.rename(columns = {'optimal_position':'best position',
                                                'oa':'overall',
                                                'pot': 'potential',
                                                'overall_value': 'value',
                                                'positional_overall': 'positional',
                                                'talent_value': 'talent',
                                                'running_value': 'running',
                                                'stealing_value': 'stealing',
                                                'ratings_value': 'ratings',
                                                'season_performance': 'season'})

    ml_pitchers = ml_pitchers.rename(columns = {'optimal_position':'best position',
                                                'oa':'overall',
                                                'pot': 'potential',
                                                'overall_value': 'value',
                                                'positional_overall': 'positional',
                                                'talent_value': 'talent',
                                                'running_value': 'running',
                                                'stealing_value': 'stealing',
                                                'ratings_value': 'ratings',
                                                'season_performance': 'season'})
    
    norm_batters = ml_batters.copy()
    norm_batters['value'] = sci.stats.zscore(ml_batters['value'])
    norm_batters['positional'] = sci.stats.zscore(ml_batters['positional'])
    norm_batters['talent'] = sci.stats.zscore(ml_batters['talent'])
    norm_batters['running'] = sci.stats.zscore(ml_batters['running'])
    norm_batters['stealing'] = sci.stats.zscore(ml_batters['stealing'])
    norm_batters['ratings'] = sci.stats.zscore(ml_batters['ratings'])
    norm_batters['season'] = sci.stats.zscore(ml_batters['season'])
    norm_batters = norm_batters.round({
        'age':0,
        'overall':0,
        'potential':0,
        'value':2,
        'positional':2,
        'talent':2,
        'running':2,
        'stealing':2,
        'ratings':2,
        'season':2
    })

    norm_pitchers = ml_pitchers.copy()
    norm_pitchers['value'] = sci.stats.zscore(ml_pitchers['value'])
    norm_pitchers['positional'] = sci.stats.zscore(ml_pitchers['positional'])
    norm_pitchers['talent'] = sci.stats.zscore(ml_pitchers['talent'])
    norm_pitchers['running'] = sci.stats.zscore(ml_pitchers['running'])
    norm_pitchers['stealing'] = sci.stats.zscore(ml_pitchers['stealing'])
    norm_pitchers['ratings'] = sci.stats.zscore(ml_pitchers['ratings'])
    norm_pitchers['season'] = sci.stats.zscore(ml_pitchers['season'])
    norm_pitchers = norm_pitchers.round({
        'age':0,
        'overall':0,
        'potential':0,
        'value':2,
        'positional':2,
        'talent':2,
        'running':2,
        'stealing':2,
        'ratings':2,
        'season':2
    })

    norm_ml = pd.concat([norm_batters,norm_pitchers], ignore_index = False)

    return norm_ml

In [29]:
def team_eval(team):
    batters = norm_batters[norm_batters['team'] == team].reset_index()
    pitchers = norm_pitchers[norm_pitchers['team'] == team].reset_index()
    return pd.concat([batters,pitchers], ignore_index = True).sort_values('season', ascending = False)


In [30]:
def pos_eval(position=''):
    data = norm_ml[['team', 'age', 'position', 'overall', 'potential', 'best position', 'positional', 'season']]
    if position == '':
        return data.sort_values('season', ascending = False).head(20)
    else:
        return data[data['position'] == position].sort_values('season', ascending = False).head(20)

In [31]:
def overall(position=''):
    data = norm_ml[['team', 'age', 'position', 'overall', 'potential', 'season','value']]
    if position == '':
        return data.sort_values('value', ascending = False).head(20)
    else:
        return data[data['position'] == position].sort_values('value', ascending = False).head(20)

In [32]:
def potential(position=''):
    data = norm_ml[['team', 'age', 'position', 'overall', 'potential', 'season','talent']]
    if position == '':
        return data.sort_values('talent', ascending = False).head(20)
    else:
        return data[data['position'] == position].sort_values('talent', ascending = False).head(20)

In [33]:
def baserunning(position=''):
    data = norm_ml[['team', 'age', 'position', 'overall', 'potential', 'season','running']]
    if position == '':
        return data.sort_values('running', ascending = False).head(20)
    else:
        return data[data['position'] == position].sort_values('running', ascending = False).head(20)

In [34]:
def stealing(position=''):
    data = norm_ml[['team', 'age', 'position', 'overall', 'potential', 'season','stealing']]
    if position == '':
        return data.sort_values('stealing', ascending = False).head(20)
    else:
        return data[data['position'] == position].sort_values('stealing', ascending = False).head(20)

In [46]:
final = dataset('PBC', 146)
final.sort_values('value', ascending = False)

,team,age,position,best position,overall,potential,value,positional,talent,running,stealing,ratings,season
player,,,,,,,,,,,,,
Jahiem Limage,San Francisco Giants,28,2B,SS,80,80,3.91,3.92,4.19,0.96,0.88,4.46,-0.01
Bart Snyder,New Orleans Voodoo,28,2B,2B,80,80,2.94,2.94,2.77,1.57,2.73,2.71,0.45
Luke Leto,San Francisco Giants,34,3B,3B,80,80,2.69,2.69,2.73,1.52,2.16,2.96,1.85
Randy Deffendall,Los Angeles Dodgers,33,SS,3B,76,76,2.56,2.56,2.78,0.87,0.59,2.96,-3.21
Joe Westberry,Mexico City Diablos,33,RF,CF,80,80,2.53,2.53,2.50,1.06,1.88,2.12,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Luis Guariento,Detroit Tigers,24,LF,LF,30,33,-2.05,-2.08,-1.49,0.78,-0.20,-1.37,-0.61
Donte Muex,Santo Domingo Iguanas,29,P,SP,40,41,-2.26,-1.83,-1.81,1.00,-0.08,-1.64,0.44
Luis Magallon,Kansas City Royals,25,SS,2B,34,34,-2.34,-2.29,-2.59,-0.90,-0.91,-2.63,0.02
